In [ ]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = sorted(glob.glob('../features/*.pkl'))
for l in fl:
    print("'{}',".format(l))

In [ ]:
choose_fl = [
'../features/other_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_5_aug1_feat.pkl',
'../features/incept_aug1_feat.pkl',
'../features/cnn_1_aug1_feat.pkl',

]

def load_x_y():
    train_x,test_x=[],[]
    for f in choose_fl:
        print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_m = LogisticRegression()
lr_m.fit(train_x,train_y)
train_pred = lr_m.predict_proba(train_x)
from sklearn.metrics import log_loss
print(log_loss(train_y,train_pred))

In [ ]:
test_df=pd.read_json('../input/test.json')
submission = pd.DataFrame()
submission['id']=test_df['id']
test_pred = lr_m.predict_proba(test_x)[:,1]
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/lr_ens_some.csv', index=False)